In [ ]:
!pip install wget

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import auth
auth.authenticate_user()

import pandas as pd
import wget
import numpy as np
import json
import os
import pickle
import yaml
import shutil
from tqdm import tqdm

os.environ['AWS_SHARED_CREDENTIALS_FILE']=path
import awscli

import matplotlib.pyplot as plt

import warnings; warnings.simplefilter('ignore')

from drive.MyDrive.mushroom_classification.new import inat_src

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
CFG = yaml.safe_load(open('drive/MyDrive/mushroom_classification/new/src/config.YAML', 'rb'))

# Downloading Data

In [ ]:
import tarfile
!aws s3 cp s3://ml-inat-competition-datasets/2021/train_mini.tar.gz .
!aws s3 cp s3://ml-inat-competition-datasets/2021/val.tar.gz .
!aws s3 cp s3://ml-inat-competition-datasets/2021/val.json.tar.gz .
!aws s3 cp s3://ml-inat-competition-datasets/2021/train_mini.json.tar.gz .

with tarfile.open('train_mini.tar.gz') as f:
    f.extractall()
with tarfile.open('val.tar.gz') as f:
    f.extractall()
with tarfile.open('val.json.tar.gz') as f:
    f.extractall()
with tarfile.open('train_mini.json.tar.gz') as f:
    f.extractall()

download: s3://ml-inat-competition-datasets/2021/train_mini.tar.gz to ./train_mini.tar.gz
download: s3://ml-inat-competition-datasets/2021/val.tar.gz to ./val.tar.gz
download: s3://ml-inat-competition-datasets/2021/val.json.tar.gz to ./val.json.tar.gz
download: s3://ml-inat-competition-datasets/2021/train_mini.json.tar.gz to ./train_mini.json.tar.gz


In [ ]:
val = src.read_aws_jsons('val.json')
train_mini = src.read_aws_jsons('train_mini.json')

remove_images_not_fungi('val', val)
remove_images_not_fungi('train_mini', train_mini)

train_mini, val = create_labels(train_mini, val)
train_mini, val = create_filepaths(train_mini, val)

In [ ]:
train_mini['gcs_filepath'] = 'gs://mushy_class/' + train_mini['filepath']
val['gcs_filepath'] = 'gs://mushy_class/' + val['filepath']

# Writing Records

In [1]:
def image_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()])
    )


def bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))


def float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def float_feature_list(value):
    """Returns a list of float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def create_example(image, path, example):
    feature = {
        "image": image_feature(image),
        "path": bytes_feature(path),
        "category_id": int64_feature(example["label"]),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))


def parse_tfrecord_fn(example):
    feature_description = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "path": tf.io.FixedLenFeature([], tf.string),
        "category_id": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, feature_description)
    example["image"] = tf.io.decode_jpeg(example["image"], channels=3)
    return example

In [2]:
rootdir = 'content'
tfrecords_dir = 'tfrecords'

train_dir = 'train_mini'
train_dir = os.path.join(rootdir, train_dir)

val_dir = 'val'
val_dir = os.path.join(rootdir, val_dir)

train_samples_per_record = 1024
train_num_records = train_mini.shape[0] // train_samples_per_record

val_samples_per_record = 256
val_num_records = val.shape[0] // val_samples_per_record

if train_mini.shape[0] % train_samples_per_record:
    train_num_records += 1
if val.shape[0] % val_samples_per_record:
    val_num_records += 1
if not os.path.exists(tfrecords_dir):
    os.makedirs(tfrecords_dir)

NameError: ignored

In [ ]:
for record_num in range(train_num_records):
    samples = train_mini.iloc[(record_num * train_samples_per_record) : ((record_num + 1) * train_samples_per_record)]
    
    with tf.io.TFRecordWriter(
        tfrecords_dir + f'/{record_num}_inat_train_{len(samples)}.tfrec'
    ) as writer:

        for idx, row in samples.iterrows():           
            img_path = row['filepath']
            image = tf.io.decode_jpeg(tf.io.read_file(img_path))
            example = create_example(image, img_path, row)
            # print(example)
            writer.write(example.SerializeToString())

In [ ]:
raw_dataset = tf.data.TFRecordDataset(f"tfrecords/1_inat_train_1024.tfrec")
parsed_dataset = raw_dataset.map(parse_tfrecord_fn)
for features in parsed_dataset.take(10):
    for key in features.keys():
        if key != "image":
            print(f"{key}: {features[key]}")

    print(f"Image shape: {features['image'].shape}")
    plt.figure(figsize=(7, 7))
    plt.imshow(features["image"].numpy())
    plt.show()

In [ ]:
for record_num in range(val_num_records):
    samples = val.iloc[(record_num * val_samples_per_record) : ((record_num + 1) * val_samples_per_record)]
    
    with tf.io.TFRecordWriter(
        tfrecords_dir + f'/val/{record_num}_inat_val_{len(samples)}.tfrec'
    ) as writer:

        for idx, row in samples.iterrows():           
            img_path = row['filepath']
            image = tf.io.decode_jpeg(tf.io.read_file(img_path))
            example = create_example(image, img_path, row)
            #print(example)
            writer.write(example.SerializeToString())

In [ ]:
raw_dataset = tf.data.TFRecordDataset(f"tfrecords/1_inat_val_256.tfrec")
parsed_dataset = raw_dataset.map(parse_tfrecord_fn)
for features in parsed_dataset.take(1):
    for key in features.keys():
        if key != "image":
            print(f"{key}: {features[key]}")

    print(f"Image shape: {features['image'].shape}")
    plt.figure(figsize=(7, 7))
    plt.imshow(features["image"].numpy())
    plt.show()

In [ ]:
!ls -l tfrecords

In [ ]:
!gsutil -m cp -r /content/tfrecords gs://mushy_class